## Import Packages

In [1]:
import io
import os
import re
import nltk 
import spacy
import pandas as pd
import docx2txt
import constants as cs
import string
import re
import os
import utils
import spacy
import pprint
from spacy.matcher import Matcher
import multiprocessing as mp 
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import warnings
warnings.filterwarnings('ignore')
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
import re
import nltk
from nltk.corpus import stopwords

### Explore pdf

In [2]:
import PyPDF2 as pdf
file = open('model_8.pdf', 'rb')
file

<_io.BufferedReader name='model_8.pdf'>

In [3]:
pdf_reader = pdf.PdfFileReader(file)
pdf_reader

### Extract the text

In [4]:
pdf_reader.getNumPages()
page1 = pdf_reader.getPage(0)
resumeText = page1.extractText()[:5000]
resumeText

'  ROBERT  SMITH   \n \n9475838374  \nEmail:  info@qwikresume.com   Website:  www.qwikresume.com  \nLinkedIn:  linkedin.com/qwikresume  Address:  1737, \nAlabama.  \nObjective   \nOver 6 years  of IT industry  experience  with 4+ years  of experience  as Mobile  application  \ndeveloper in the field of Android. Experience in developing front end applications for Android  \nphones.  Experience  developing  mobile  applications  on Android  platform,  building  Custom  UI using \nViews,  ViewGroups,  Layouts,  Widgets  and graphics  that scale  based  on the screen  size using  9- \npatch  images,  localization, testing  and publishing  the applications  to the Android  Market.  \n \nSkills   \nPython,  Java,  C, Javascript,  Matlab,  R. \nDomain . \nJava  Developer . \n \n \nWork  Experience   \nABC Corporation  \xad January  2011  – March  2012  \n• Environment  Eclipse  IDE, Android  Studio,  GenyMotion,  Java,  Android  SDK, Android  \nDevelopment Tools (ADT), JSON, XML, \n• Involved

### NLP : Regular Expression

In [5]:
def cleanResume(resumeText):                                                                   #NLP : REG EXP
    #resumeText = re.sub('http\S+\s*', ' ', resumeText)  # remove URLs
   # resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
    #resumeText = re.sub(r'\n', ',', resumeText)
    resumeText = re.sub('#\S+', '', resumeText)  # remove hashtags
   # resumeText = re.sub('@\S+', '  ', resumeText)  # remove mentions
   # resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resumeText)  # remove punctuations
   # resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*,-/<=>?[]^`{|}~"""), ' ', resumeText)
    resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText) 
    resumeText = re.sub('\s+', ' ', resumeText)  # remove extra whitespace
    return resumeText
clean = cleanResume(resumeText)
clean

' ROBERT SMITH 9475838374 Email: info@qwikresume.com Website: www.qwikresume.com LinkedIn: linkedin.com/qwikresume Address: 1737, Alabama. Objective Over 6 years of IT industry experience with 4+ years of experience as Mobile application developer in the field of Android. Experience in developing front end applications for Android phones. Experience developing mobile applications on Android platform, building Custom UI using Views, ViewGroups, Layouts, Widgets and graphics that scale based on the screen size using 9- patch images, localization, testing and publishing the applications to the Android Market. Skills Python, Java, C, Javascript, Matlab, R. Domain . Java Developer . Work Experience ABC Corporation January 2011 March 2012 Environment Eclipse IDE, Android Studio, GenyMotion, Java, Android SDK, Android Development Tools (ADT), JSON, XML, Involved in the full life cycle of the project including analysis design, development, debugging, testing, and deployment. Developed the appl

In [6]:
clean = " ".join(clean.split())
clean

'ROBERT SMITH 9475838374 Email: info@qwikresume.com Website: www.qwikresume.com LinkedIn: linkedin.com/qwikresume Address: 1737, Alabama. Objective Over 6 years of IT industry experience with 4+ years of experience as Mobile application developer in the field of Android. Experience in developing front end applications for Android phones. Experience developing mobile applications on Android platform, building Custom UI using Views, ViewGroups, Layouts, Widgets and graphics that scale based on the screen size using 9- patch images, localization, testing and publishing the applications to the Android Market. Skills Python, Java, C, Javascript, Matlab, R. Domain . Java Developer . Work Experience ABC Corporation January 2011 March 2012 Environment Eclipse IDE, Android Studio, GenyMotion, Java, Android SDK, Android Development Tools (ADT), JSON, XML, Involved in the full life cycle of the project including analysis design, development, debugging, testing, and deployment. Developed the appli

### NLP : Tokenize[Word Tokenize]

In [14]:
from nltk.tokenize import word_tokenize                                                             #NLP : Tokenize
tokens = word_tokenize(clean)
print(tokens)
tok_name = set(tokens)

['ROBERT', 'SMITH', '9475838374', 'Email', ':', 'info', '@', 'qwikresume.com', 'Website', ':', 'www.qwikresume.com', 'LinkedIn', ':', 'linkedin.com/qwikresume', 'Address', ':', '1737', ',', 'Alabama', '.', 'Objective', 'Over', '6', 'years', 'of', 'IT', 'industry', 'experience', 'with', '4+', 'years', 'of', 'experience', 'as', 'Mobile', 'application', 'developer', 'in', 'the', 'field', 'of', 'Android', '.', 'Experience', 'in', 'developing', 'front', 'end', 'applications', 'for', 'Android', 'phones', '.', 'Experience', 'developing', 'mobile', 'applications', 'on', 'Android', 'platform', ',', 'building', 'Custom', 'UI', 'using', 'Views', ',', 'ViewGroups', ',', 'Layouts', ',', 'Widgets', 'and', 'graphics', 'that', 'scale', 'based', 'on', 'the', 'screen', 'size', 'using', '9-', 'patch', 'images', ',', 'localization', ',', 'testing', 'and', 'publishing', 'the', 'applications', 'to', 'the', 'Android', 'Market', '.', 'Skills', 'Python', ',', 'Java', ',', 'C', ',', 'Javascript', ',', 'Matlab',

### NLP : POS Tagging

In [15]:
import spacy                                                                                                        #Name
import en_core_web_sm
from spacy.matcher import Matcher

# load pre-trained model
nlp = en_core_web_sm.load()

# initialize matcher with a vocab
matcher = Matcher(nlp.vocab)


def extract_name(resume_text):
    nlp_text = nlp(resume_text)
    
    # First name and Last name are always Proper Nouns
    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
    
    matcher.add('NAME', [pattern], on_match=None)
    
    matches = matcher(nlp_text)
    
    for match_id, start, end in matches:
        span = nlp_text[start:end]
        return span.text
name = extract_name(clean)
print('Name: ',name)

Emp_name = name                                                                                #Get a name 

Name:  ROBERT SMITH


In [9]:
lis = []                                                                                                        #Skills

for word in tok_name:
    lis.append(word.lower())
str = lis

import Values as v
a = set(str)
b = set(v.str1)
c = []

for i in a:
    if i in b:
        c.append(i)
print("Skills",c)

Emp_skill =  f'"{c}"'                                                          #Skill

Skills ['python', 'code', 'matlab', 'c', 'javascript', 'java']


### NLP : Regular Expression

In [10]:
from pdfminer.high_level import extract_text                                                      #EMAIL
 
EMAIL_REG = re.compile(r'[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+')
def extract_emails(text):
    return re.findall(EMAIL_REG, clean)
text = clean
emails = extract_emails(clean)
 
if emails:
    mail = emails[0]
    print(mail)
    
Emp_mail = mail

info@qwikresume.com


### NLP : Regular Expression

In [11]:
def extract_mobile_number(resume_text):                                                      #Mobile
    phone = re.findall(re.compile(r'(?:(?:\+?([1-9]|[0-9][0-9]|[0-9][0-9][0-9])\s*(?:[.-]\s*)?)?(?:\(\s*([2-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9])\s*\)|([0-9][1-9]|[0-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9]))\s*(?:[.-]\s*)?)?([2-9]1[02-9]|[2-9][02-9]1|[2-9][02-9]{2})\s*(?:[.-]\s*)?([0-9]{4})(?:\s*(?:#|x\.?|ext\.?|extension)\s*(\d+))?'), resume_text)
    
    if phone:
        number = ''.join(phone[0])
        if len(number) > 10:
            return number
        else:
            return number
mobile = extract_mobile_number(clean)
print('Mobile Number: ',mobile)

Emp_Mobile = mobile

Mobile Number:  9475838374


### NLP : Tokenize[Sentence Tokenize]

In [12]:
from nltk.tokenize import sent_tokenize                                                             #NLP : Tokenize
token = sent_tokenize(clean)
print(token)

['ROBERT SMITH 9475838374 Email: info@qwikresume.com Website: www.qwikresume.com LinkedIn: linkedin.com/qwikresume Address: 1737, Alabama.', 'Objective Over 6 years of IT industry experience with 4+ years of experience as Mobile application developer in the field of Android.', 'Experience in developing front end applications for Android phones.', 'Experience developing mobile applications on Android platform, building Custom UI using Views, ViewGroups, Layouts, Widgets and graphics that scale based on the screen size using 9- patch images, localization, testing and publishing the applications to the Android Market.', 'Skills Python, Java, C, Javascript, Matlab, R. Domain .', 'Java Developer .', 'Work Experience ABC Corporation January 2011 March 2012 Environment Eclipse IDE, Android Studio, GenyMotion, Java, Android SDK, Android Development Tools (ADT), JSON, XML, Involved in the full life cycle of the project including analysis design, development, debugging, testing, and deployment.'

### Named Entity Recognition

In [13]:
###Edites
import Values as n
import pandas as pd
import spacy
nlp = spacy.load("en_core_web_sm")  
doc = nlp(clean)    
for chunk in doc.noun_chunks:      
    noun_chunks = chunk.text

def extract_skills(resume_text):
    nlp_text = nlp(resume_text)

    # removing stop words and implementing word tokenization
    tokens = [token.text for token in nlp_text if not token.is_stop]
    
    # reading the csv file
   
    
    lis = []                                                                                                        #Skills


    str = token

    import Values as v
    a = set(str)
    b = set(v.noun_chunk)
    c = []

    for i in a:
        if i in b:
            c.append(i)
    return c


   
a = f'"{extract_skills(clean)}"'
print("Domain",a)


Domain "['Java Developer .']"


In [14]:
import re
Emp_Domain = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', a)
Emp_Skill = re.sub('[%s]' % re.escape("""!"#$%&'()*+-./:;<=>?@[\]^_`{|}~"""), ' ', Emp_skill)


In [15]:
stop = "!"
def Reg(a,b,c,d,e,f):
    a = re.sub('\s+', ' ', a)
    b = re.sub('\s+', ' ', b)
    c = re.sub('\s+', ' ', c)
    d = re.sub('\s+', ' ', d)
    e = re.sub('\s+', ' ', e)
    a =" ".join(a.split())
    b =" ".join(b.split())
    c =" ".join(c.split())
    d =" ".join(d.split())
    e =" ".join(e.split())
    f = "!"
    return a,b,c,d,e,f
   

print(Reg(Emp_name,Emp_Domain,Emp_Skill,Emp_mail,Emp_Mobile,stop))



    


('ROBERT SMITH', 'Java Developer', 'python , code , matlab , c , javascript , java', 'info@qwikresume.com', '9475838374', '!')


### After Extracting all

In [16]:
print(Reg(Emp_name,Emp_Domain,Emp_Skill,Emp_mail,Emp_Mobile,stop))

('ROBERT SMITH', 'Java Developer', 'python , code , matlab , c , javascript , java', 'info@qwikresume.com', '9475838374', '!')


### Import the details in csv format

In [17]:
from csv import DictWriter
field_names = ['Category', 'Resume']
dict = {'Category': Emp_Domain, 'Resume':[Emp_name,Emp_Domain,Emp_Skill,Emp_mail,Emp_Mobile,stop]}
with open('Emp.csv', 'a') as f_object:
	dictwriter_object = DictWriter(f_object, fieldnames=field_names)

	
	dictwriter_object.writerow(dict)

	
	f_object.close()


In [18]:
print("Name : ",Emp_name)
print("Domain : ",Emp_Domain)
print("Skills : ",Emp_Skill)
print("Mail : ",Emp_mail)
print("Phone no : ",Emp_Mobile)

Name :  ROBERT SMITH
Domain :     Java Developer     
Skills :     python ,  code ,  matlab ,  c ,  javascript ,  java   
Mail :  info@qwikresume.com
Phone no :  9475838374
